# Connect Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/TTKH

/content/drive/MyDrive/TTKH


# Import Lib

In [ ]:
pip install ipywidgets

In [ ]:

import math
from abc import ABC, abstractmethod
import numpy as np
import random as random
from math import pi, cos, sin, sqrt
import matplotlib.pyplot as plt
from IPython import display
from time import sleep
import ipywidgets  as widgets
from ipywidgets import interact, interact_manual,Layout,HBox,VBox,interactive


from  Configs import Configs
from DirectionalBias import DirectionalBias
from GeneralDLA2 import GeneralDLA2
from ConstantStickDistribution import ConstantStickDistribution

# Main Class


In [ ]:
class Main:
  def display(self,dla):
    plt.figure(figsize=(Configs.figure_size,Configs.figure_size))
    extent = 0, Configs.window_size,0, Configs.window_size

    white_cell_value = 0

    masked_array = np.ma.masked_where(dla.surface_matrix == white_cell_value, dla.surface_matrix)

    # plt.clf()
    # plt.cla()
    plt.imshow(masked_array,extent = extent,origin ='lower')
    display.clear_output(wait=True)
    display.display(plt.gcf())
    plt.clf()
    plt.cla()
    plt.close()
    sleep(0.1)


  def simulate(self):
    dla=GeneralDLA2()
    current_stage=dla.transformValue()
    while dla.getRealBindedParticles()<=Configs.num_particles:
      dla.iterate()
      if dla.transformValue() != current_stage:
        current_stage= dla.transformValue();
        print("Current_Stage :",current_stage)
        self.display(dla)
    print("Đã chạy xong")


  def stop(self):
    print("Stop")
    self.is_simulate=False;
    print("self.is_simulate",self.is_simulate)

In [ ]:
executor=Main()

# Visualize

In [ ]:
width_size= '60%'
description_width= '40%'

figure_size_ui=widgets.IntSlider(
    value=10,
    min=5,
    max=12,
    step=1,
    layout=Layout(width=width_size),
    description='Kích thước trục tọa độ( figure_size)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': description_width,'description_align':'center','width':'100%'},
    readout=True,
    readout_format='d'
)

color_num_ui=widgets.IntSlider(
    value=5,
    min=1,
    max=25,
    step=1,
    layout=Layout(width=width_size),
    description='Số giai đoạn trích xuất:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width':description_width,'width':'100%'},
    readout=True,
    readout_format='d'
)

window_size_ui=widgets.IntSlider(
    value=20,
    min=10,
    max=200,
    step=5,
    layout=Layout(width=width_size),
    style={'description_width': description_width},
    description='Kích thước cửa sổ (số ô/trục)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


distribution_ui=widgets.RadioButtons(
    options=[('Hằng số','constant'),('Dựa vào số lượng hàng xóm','neightbor'), ('Dựa vào vị trí tiếp xúc','curve')],
    value='constant',
    description='Loại xác suất dính:',
    layout=Layout(width=width_size),
    style={'description_width': description_width,'width':'100%'},
    #disabled=True
)

add_cluster_toward_force_ui=widgets.RadioButtons(
    options=[('Có',True),('Không',False)],
    value=True,
    description='Thêm lực hướng tinh thể : ',
    layout=Layout(width=width_size),
    style={'description_width': description_width,'width':'100%'},
    #disabled=True
)


const_prob_ui=widgets.FloatSlider(
    value=0.60,
    min=0.01,
    max=1.00,
    step=0.01,
    description='Xác suất hằng số dính:',
    layout=Layout(width=width_size),
    style={'description_width': description_width,'width':'100%'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.01f',
)

def changeConstProb(*args):
    if distribution_ui.value == 'constant':
      const_prob_ui.min = 0.01
      const_prob_ui.max=1
      const.step=0.01
      const_prob_ui.value=0.7
    else:
      const_prob_ui.min = 0.7
      const_prob_ui.max=0.7
      const_prob_ui.value=0.7

distribution_ui.observe(changeConstProb,'value')

direction_bias_ui=widgets.Dropdown(
    options=[('Sang cạnh trái','left'),('Sang cạnh phải','right'),('Xuống cạnh dưới','bottom'),('Lên cạnh trên','top'),\
             ('Đường xích đạo','equator'),('Đường kinh tuyến','meridian'),('Hướng tâm','center'),('Hướng về các cạnh','edges')],
    value='center',
    layout=Layout(width=width_size),
    style={'description_width': description_width},
    description='Vị trí của tinh thể ban đầu:'
)

num_particles_ui=widgets.IntSlider(
    value=200,
    min=10,
    max=5000,
    step=10,
    layout=Layout(width=width_size), 
    description='Số hạt cần kết tụ:',
    style={'description_width': description_width},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

num_walkers_ui=widgets.IntSlider(
    value=5,
    min=1,
    max=500,
    step=10,
    layout=Layout(width=width_size),
    description='Số hạt di chuyển đồng thời:',
    style={'description_width': description_width},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)



force_bias_ui=widgets.IntSlider(
    value=1,
    min=1,
    max=100,
    layout=Layout(width=width_size),
    step=1,
    description='Chiều dài bước tối đa của hạt:',
    style={'description_width': description_width},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


def visualizeDLA(figure_size,window_size,color_num,\
                 distribution,const_prob,\
                 direction_bias,add_cluster_toward_force,force_bias,\
                 num_particles,num_walkers):
  
  if distribution == 'constant':
    Configs.distribution=ConstantStickDistribution(proba=const_prob)
  else:
    Configs.distribution=ConstantStickDistribution(proba=const_prob) 
  Configs.add_cluster_toward_force=add_cluster_toward_force;
  Configs.color_number=color_num
  Configs.direction_bias=DirectionalBias(direction_bias)
  Configs.force_bias=force_bias
  Configs.figure_size=figure_size
  Configs.window_size=window_size
  Configs.num_particles=num_particles
  Configs.num_walkers=num_walkers
  executor.simulate()

visual_ui = interact_manual(visualizeDLA,figure_size=figure_size_ui,window_size=window_size_ui,\
                    distribution=distribution_ui,direction_bias=direction_bias_ui,add_cluster_toward_force= add_cluster_toward_force_ui,\
                    const_prob=const_prob_ui,color_num=color_num_ui,\
                     num_particles=num_particles_ui,num_walkers=num_walkers_ui,force_bias=force_bias_ui)


interactive(children=(IntSlider(value=10, continuous_update=False, description='Kích thước trục tọa độ( figure…